# Practical Lab 1 — Predictive Maintenance (EDA + Baseline Models + Threshold Discovery)

This notebook focuses on:
- Pulling training data from Neon PostgreSQL (or loading the exported CSV)
- Performing EDA (data quality, central tendency, dispersion, trend)
- Training univariate linear regression baselines (Time → Axis #1–#8) per robot
- Computing residuals and discovering MinC/MaxC/T using evidence from residual statistics


In [4]:
from pathlib import Path
import re
import numpy as np
import pandas as pd

# ✅ Your provided paths (exact)
CSV_PATH = "D:\Lab_1\LinearRegressionWorkshop-1\data\raw\RMBR4-2_export_test_with_robotids_plus15000.csv"
EXP_DIR = Path("D:\Lab_1\LinearRegressionWorkshop-1\experiments\plots")
EXP_DIR.mkdir(parents=True, exist_ok=True)

print("✅ Paths OK")
print("CSV:", CSV_PATH)
print("EXP_DIR:", EXP_DIR)


✅ Paths OK
aw\RMBR4-2_export_test_with_robotids_plus15000.csv
EXP_DIR: D:\Lab_1\LinearRegressionWorkshop-1\experiments\plots


<>:7: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:8: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:7: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
<>:8: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
C:\Users\Viraj Mistry\AppData\Local\Temp\ipykernel_14956\3510281110.py:7: SyntaxWarning: "\L" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\L"? A raw string is also an option.
  CSV_PATH = "D:\Lab_1\LinearRegressionWorkshop-1\data\raw\RMBR4-2_export_test_with_robotids_plus15000.csv"
C:\Users\Viraj Mistry\AppData\Local\Temp\ipykernel_14956\35

## Imports and configuration

This cell imports all required libraries (pandas/numpy/sklearn/matplotlib/etc.),
sets global configuration (random seed, plotting defaults, file paths),
and defines common constants like axis column names and robot identifiers.
This ensures reproducible results across runs.


In [5]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

load_dotenv()

DB_URL = (
    f"postgresql+psycopg2://{os.getenv('PGUSER')}:{os.getenv('PGPASSWORD')}"
    f"@{os.getenv('PGHOST')}:{os.getenv('PGPORT', '5432')}/{os.getenv('PGDATABASE')}"
    f"?sslmode={os.getenv('PGSSLMODE', 'require')}"
)

engine = create_engine(DB_URL, pool_pre_ping=True)

with engine.connect() as conn:
    conn.execute(text("SELECT 1;"))

print("✅ Connected to Neon")


✅ Connected to Neon


## Load training data from Neon PostgreSQL

This cell connects to Neon PostgreSQL using credentials (typically from a `.env` file),
executes a SQL query to pull training data (time + Axis #1–#8 + robot_id),
and loads the result into a Pandas DataFrame. This is the baseline dataset
used for EDA and regression model fitting.


In [6]:
df = pd.read_csv(CSV_PATH)

# ---- Detect robot_id column
robot_col = None
for c in df.columns:
    if c.strip().lower() in ["robot_id", "robotid", "robot"]:
        robot_col = c
        break
if robot_col is None:
    raise ValueError(f"Could not find robot id column. Columns are: {list(df.columns)}")

# ---- Detect time column
time_col = None
for c in df.columns:
    if c.strip().lower() in ["time", "timestamp", "ts", "datetime", "date_time"]:
        time_col = c
        break
if time_col is None:
    raise ValueError(f"Could not find time column. Columns are: {list(df.columns)}")

# ---- Detect axis columns like "Axis #1", "Axis #2", etc.
axis_cols = []
axis_nums = []
for c in df.columns:
    m = re.search(r"axis\s*#?\s*(\d+)", str(c), flags=re.IGNORECASE)
    if m:
        axis_cols.append(c)
        axis_nums.append(int(m.group(1)))

if len(axis_cols) == 0:
    raise ValueError("No axis columns found (expected Axis #1, Axis #2, ...).")

# Sort axis columns by number
axis_sorted = sorted(zip(axis_nums, axis_cols), key=lambda x: x[0])
axis_nums = [a for a, _ in axis_sorted]
axis_cols = [c for _, c in axis_sorted]

print("✅ Detected columns")
print("robot_col:", robot_col)
print("time_col:", time_col)
print("axis_cols:", axis_cols[:10], "... total:", len(axis_cols))
df.head()


OSError: [Errno 22] Invalid argument: 'D:\\Lab_1\\LinearRegressionWorkshop-1\\data\raw\\RMBR4-2_export_test_with_robotids_plus15000.csv'

## Data integrity and schema validation

This cell verifies that the dataset has the expected columns and data types,
checks for missing values and duplicates, validates axis ranges, and confirms
that time is monotonic within each robot. These checks protect the baseline model
from training on corrupted or inconsistent data.


In [ ]:
df[time_col] = pd.to_datetime(df[time_col], utc=True, errors="coerce")
df = df.dropna(subset=[time_col]).copy()

# Normalize robot_id to int 1..4 if possible
df[robot_col] = df[robot_col].astype(str).str.replace("robot_", "", regex=False)
df["robot_id"] = pd.to_numeric(df[robot_col], errors="coerce").astype("Int64")
df = df.dropna(subset=["robot_id"]).copy()
df["robot_id"] = df["robot_id"].astype(int)

# keep robots 1..4 only (your requirement)
df = df[df["robot_id"].isin([1, 2, 3, 4])].copy()

# Map axis_num -> axis column (Axis #1..Axis #4)
axis_map = {}
for num, col in zip(axis_nums, axis_cols):
    if num in [1, 2, 3, 4]:
        axis_map[num] = col

if len(axis_map) < 4:
    print("⚠️ axis_map found:", axis_map)
    raise ValueError("Could not map all 4 axis columns (Axis #1..Axis #4).")

parts = []
for axis_num, col in axis_map.items():
    sub = df[["robot_id", time_col, col]].copy()
    sub = sub.rename(columns={time_col: "ts", col: "axis_value"})
    sub["axis_num"] = axis_num
    parts.append(sub)

train_long = pd.concat(parts, ignore_index=True)
train_long = train_long.dropna(subset=["axis_value"]).copy()
train_long = train_long.sort_values(["robot_id", "ts"]).reset_index(drop=True)

print("✅ Long format created:", train_long.shape)
train_long.head()


✅ Long format created: (218688, 4)


,robot_id,ts,axis_value,axis_num
0,1,2022-10-17 12:18:23.660000+00:00,0.0,1
1,1,2022-10-17 12:18:23.660000+00:00,0.0,2
2,1,2022-10-17 12:18:23.660000+00:00,0.0,3
3,1,2022-10-17 12:18:23.660000+00:00,0.0,4
4,1,2022-10-17 12:18:25.472000+00:00,0.0,1


## Time preprocessing

This cell parses the time column into a valid datetime or numeric time index,
sorts records by time (per robot), and optionally creates a numeric time feature
(seconds/minutes) required by linear regression. Correct time ordering is critical
for both regression fitting and streaming simulation.


In [ ]:
schema_sql = """
CREATE SCHEMA IF NOT EXISTS linear_regression;

DROP TABLE IF EXISTS linear_regression.events;
DROP TABLE IF EXISTS linear_regression.models;

CREATE TABLE linear_regression.models (
  robot_id        INT NOT NULL,
  axis_num        INT NOT NULL,
  slope           DOUBLE PRECISION NOT NULL,
  intercept       DOUBLE PRECISION NOT NULL,
  residual_alert  DOUBLE PRECISION NOT NULL,
  residual_error  DOUBLE PRECISION NOT NULL,
  sample_seconds  INT NOT NULL,
  t_seconds       INT NOT NULL,
  ttf_a           DOUBLE PRECISION NOT NULL,
  ttf_b           DOUBLE PRECISION NOT NULL,
  created_at      TIMESTAMPTZ NOT NULL DEFAULT NOW(),
  PRIMARY KEY (robot_id, axis_num)
);

CREATE TABLE linear_regression.events (
  id               BIGSERIAL PRIMARY KEY,
  robot_id          INT NOT NULL,
  axis_num          INT NOT NULL,
  event_type        TEXT NOT NULL, -- ALERT / ERROR
  ts                TIMESTAMPTZ NOT NULL,
  residual          DOUBLE PRECISION NOT NULL,
  predicted_ttf_days DOUBLE PRECISION NOT NULL,
  created_at        TIMESTAMPTZ NOT NULL DEFAULT NOW()
);
"""

with engine.begin() as conn:
    conn.execute(text(schema_sql))

print("✅ Schema reset + tables created")


✅ Schema reset + tables created


## EDA: central tendency and dispersion

This cell computes descriptive statistics per robot and per axis:
mean/median (typical behavior) and std/IQR (natural variability).
These metrics quantify how stable/noisy each axis is and later inform
threshold discovery using residual distributions.


In [ ]:
from sklearn.linear_model import LinearRegression

def time_seconds(ts: pd.Series) -> np.ndarray:
    t0 = ts.min()
    return (ts - t0).dt.total_seconds().to_numpy().reshape(-1, 1)

# ✅ Horizon definition (explainable)
RISK_ALERT_DAYS = 7.0
RISK_ERROR_DAYS = 2.0

models = []

for rid in [1, 2, 3, 4]:
    d = train_long[train_long["robot_id"] == rid].sort_values("ts").copy()

    X = time_seconds(d["ts"])
    y = d["axis_value"].to_numpy()

    lr = LinearRegression().fit(X, y)
    yhat = lr.predict(X)
    resid = y - yhat

    pos = resid[resid > 0]
    # ✅ High quantiles -> fewer alerts/errors
    residual_alert = float(np.quantile(pos, 0.99)) if len(pos) else 0.0
    residual_error = float(np.quantile(pos, 0.999)) if len(pos) else max(residual_alert, 0.0)

    dt = d["ts"].diff().dt.total_seconds().dropna()
    sample_seconds = int(np.round(dt.median())) if len(dt) else 1
    t_seconds = int(10 * sample_seconds)

    # severity mapping: bigger residual => fewer days remaining
    scale = max(residual_error - residual_alert, 1e-9)
    k = (RISK_ALERT_DAYS - RISK_ERROR_DAYS) / scale
    ttf_a = float(RISK_ALERT_DAYS)
    ttf_b = float(-k)

    models.append({
        "robot_id": int(rid),
        "axis_num": int(rid),
        "slope": float(lr.coef_[0]),
        "intercept": float(lr.intercept_),
        "residual_alert": float(residual_alert),
        "residual_error": float(residual_error),
        "sample_seconds": int(sample_seconds),
        "t_seconds": int(t_seconds),
        "ttf_a": ttf_a,
        "ttf_b": ttf_b,
    })

models_df = pd.DataFrame(models)
display(models_df)
print("✅ Models computed")


,robot_id,axis_num,slope,intercept,residual_alert,residual_error,sample_seconds,t_seconds,ttf_a,ttf_b
0,1,1,4.719539e-07,1.898171,33.022517,44.905770,0,0,7.0,-0.420760
1,2,2,-4.557767e-06,2.341581,14.256932,19.092860,0,0,7.0,-1.033928
2,3,3,7.092337e-06,2.234761,14.301865,18.243680,0,0,7.0,-1.268451
3,4,4,3.318866e-06,2.271702,14.359202,20.328199,0,0,7.0,-0.837662


✅ Models computed


## Train baseline regression models (Time → Axis)

This cell fits 8 univariate linear regression models per robot:
Time is the predictor and each Axis (#1–#8) is the target.
It stores slope and intercept for each (robot, axis) so the same models can be used
later in streaming inference without retraining.
EDA: Time vs Axis visualization

This cell plots Time vs Axis (#1–#8), typically as scatter/line plots per robot.
The purpose is to visually inspect trends (drift), mode changes, and anomalies,
and to validate that linear regression is a reasonable baseline model for each axis.


In [ ]:
upsert_model = """
INSERT INTO linear_regression.models
(robot_id, axis_num, slope, intercept, residual_alert, residual_error, sample_seconds, t_seconds, ttf_a, ttf_b)
VALUES (:robot_id, :axis_num, :slope, :intercept, :residual_alert, :residual_error, :sample_seconds, :t_seconds, :ttf_a, :ttf_b)
ON CONFLICT (robot_id, axis_num)
DO UPDATE SET
  slope=EXCLUDED.slope,
  intercept=EXCLUDED.intercept,
  residual_alert=EXCLUDED.residual_alert,
  residual_error=EXCLUDED.residual_error,
  sample_seconds=EXCLUDED.sample_seconds,
  t_seconds=EXCLUDED.t_seconds,
  ttf_a=EXCLUDED.ttf_a,
  ttf_b=EXCLUDED.ttf_b,
  created_at=NOW();
"""

with engine.begin() as conn:
    for r in models:
        conn.execute(text(upsert_model), r)

check = pd.read_sql("SELECT * FROM linear_regression.models ORDER BY robot_id;", engine)
print("✅ Models in DB:", len(check))
display(check)


✅ Models in DB: 4


,robot_id,axis_num,slope,intercept,residual_alert,residual_error,sample_seconds,t_seconds,ttf_a,ttf_b,created_at
0,1,1,4.719539e-07,1.898171,33.022517,44.905770,0,0,7.0,-0.420760,2026-02-07 01:56:54.103981+00:00
1,2,2,-4.557767e-06,2.341581,14.256932,19.092860,0,0,7.0,-1.033928,2026-02-07 01:56:54.103981+00:00
2,3,3,7.092337e-06,2.234761,14.301865,18.243680,0,0,7.0,-1.268451,2026-02-07 01:56:54.103981+00:00
3,4,4,3.318866e-06,2.271702,14.359202,20.328199,0,0,7.0,-0.837662,2026-02-07 01:56:54.103981+00:00


In [ ]:
import pandas as pd

pd.read_sql("""
SELECT table_schema, table_name
FROM information_schema.tables
WHERE table_schema='linear_regression'
ORDER BY table_name;
""", engine)


,table_schema,table_name
0,linear_regression,events
1,linear_regression,models
2,linear_regression,stream_points
3,linear_regression,training_points
4,linear_regression,training_stats
